<a href="https://colab.research.google.com/github/Kimhansav/everynocode_search_engine/blob/main/BP_make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#KcBERT 학습용 데이터셋을 제작하는 코드(로컬)
#제작할 데이터셋은 pretraining용 데이터셋(soynlp 학습용 데이터), finetuning용 데이터셋(next sentence prediction)

In [87]:
# !pip install kss
!pip install datasets
import pandas as pd
import numpy as np
import re
import os
import torch
import tensorflow as tf
import urllib.request
# from kss import Kss
from tqdm import tqdm
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import AutoTokenizer, shape_list, TFBertModel, RobertaTokenizerFast, RobertaForSequenceClassification, TextClassificationPipeline, pipeline, BertTokenizer, BertForNextSentencePrediction,  TrainingArguments
from datasets import Dataset, load_dataset
from sklearn.model_selection import StratifiedKFold, train_test_split
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
#전처리된 파일 로드(BP_spacing 건너뛰기)
talk_save_path = '/content/drive/My Drive/talk_preprocess_result_short.xlsx'
qna_path = '/content/drive/My Drive/community_qna_preprocessed.xlsx'
all_contents_path = '/content/drive/My Drive/community_all_contents_preprocessed.csv'
all_comments_path = '/content/drive/My Drive/community_all_comments_preprocessed.csv'

df_talk = pd.read_excel(talk_save_path)
df_qna = pd.read_excel(qna_path)
df_all_contents = pd.read_csv(all_contents_path)
df_all_comments = pd.read_csv(all_comments_path)

In [112]:
#직접 제작한 카카오톡 질문답변 데이터셋 로드
talk_finetune_path = '/content/drive/My Drive/talk_finetune_dataset.xlsx'

df_talk_finetune = pd.read_excel(talk_finetune_path, index_col = 0)

#Pretrain 데이터셋, 제공받은 모든 데이터 사용


In [ ]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def seperate_to_sentences(corpus_list):
  new_corpus_list = []
  for corpus in corpus_list:
    new_corpus_list.append(kss.split_sentences(corpus))
  new_corpus_list = sum(lists for lists in new_corpus_list)
  return new_corpus_list

In [ ]:
corpus_list = ['딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?', '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?', '딥 러닝 자연어 처리가 재미있기는 합니다. 그런데 문제는 영어보다 한국어로 할 때 너무 어렵습니다. 이제 해보면 알걸요?']
print(seperate_to_sentences(corpus_list))

In [ ]:
#kss로 리스트 안에 있는 여러 문장으로 이루어진 글을 문장 단위로 분리하는 함수
def separate_to_sentences_and_deduplicate(DataFrame, columns):
    sentences_list = []
    for column in columns: #데이터프레임의 각 열에 대해서 처리
        text_list = DataFrame[column].to_list()
        sentences_list.append(sum(kss.split_sentences(text) for text in text_list))
    return list(set(sum(lists for lists in sentences_list)))

In [ ]:
#데이터프레임의 열들을 리스트로 변환
talk_list = separate_to_sentences_and_deduplicate(df_talk, ['text'])

In [ ]:
qna_list = separate_to_sentences_and_deduplicate(df_qna, ['내용', '답변', '댓글'])

In [ ]:
all_contents_list = separate_to_sentences_and_deduplicate(df_all_contents, ['내용', '답변', '댓글'])

In [ ]:
all_comments_list = separate_to_sentences_and_deduplicate(df_all_comments, ['내용', '대댓글', '댓글'])

In [ ]:
#하나의 리스트로 결합
df_result = pd.DataFrame(sum(talk_list, qna_list, all_contents_list, all_comments_list), index = ['text']).T
print(len(df_result))

In [ ]:
#.xlsx 파일로 저장
pretrain_data_save_path = '/content/drive/My Drive/KcBERT_pretrain_dataset.xlsx'

df_result.to_excel(pretrain_data_save_path)

#Finetune 데이터셋


In [113]:
#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[pd.isna(df_qna['Slug']) == False]
df_question_answer = df_question_answer[pd.isna(df_question_answer['답변']) == False]

print(df_question_answer)
print(len(df_question_answer))

print(df_question_answer.iloc[0])
print(df_question_answer.iloc[1]['답변'])

     index                                                 내용  \
0        0  안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
2        2  안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
4        4  안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
9        9  모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
11      11  한번에 여러개의 파일을 업로드하는 기능을 구현하기 위해서 버블 무료 플러그인 Dro...   
..     ...                                                ...   
542    542  스포츠 클럽을 만들고 클럽원들의 신청을 받아서 허가해주고, 클럽원들의 매치 기록을 ...   
545    545  custom state 만들어서 하고 있는데 파란색이지만  오류가 나고 있어요당연히...   
549    549  정렬 기능을 사용중인데아래의 그림과 같이 2가지의 문제가 해결되지 않고 있습니다 ....   
558    558  버블에서 아래 UI처럼 +버튼을 누르면 하위에 계속해서 폴더가 추가되도록 프론트를 ...   
575    575  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                    답변  \
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...   
2    헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...   
4    충분히 버블로 개발가능할 것 같아요! 이미 비슷한 수준의 어플을 개발한 

질문 + 답변에다가 답변에 달린 댓글들까지 입력 데이터로 처리할 경우 퀄리티가 더 좋아질 수 있지만, max_length를 초과하지 않는 방향에 따라 답변 데이터만 이용, 나중에 개선방안 찾아보기

community_qna_preprocessed 데이터에서 이미지 파일을 base64 인코딩한 텍스트가 섞여 있는데, 데이터 파일 추출 과정에서 직접 제거하는 게 좋을 것 같음.

In [114]:
#커뮤니티 파일을 가공해서 finetune 데이터셋을 제작
finetune_dict = {}

#slug가 있다면 ['내용'] == 질문
df_question_answer = df_qna[pd.isna(df_qna['Slug']) == False]
df_question_answer = df_question_answer[pd.isna(df_question_answer['답변']) == False]

#['답변']의 텍스트를 ' , ' 로 구분 후 질문에 하나씩 쌓기
sent1_list, sent2_list, label_list, index_list = [], [], [], []
for i in range(len(df_question_answer)):
  row = df_question_answer.iloc[i]
  question_text = row['내용']
  answer_text_list = row['답변'].split(' , ')
  for j in range(len(answer_text_list)):
    sent1_list.append(question_text + ''.join(answer_text_list[:j]))
    sent2_list.append(answer_text_list[j])
    label_list.append(1)
    index_list.append(row['index'])

finetune_dict = {'sent1' : sent1_list, 'sent2' : sent2_list, 'label' : label_list, 'sent2_index' : index_list}

df_community_finetune = pd.DataFrame.from_dict(finetune_dict)
print(df_community_finetune)

                                                 sent1  \
0    안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2    안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3    안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
4    모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
..                                                 ...   
227  custom state 만들어서 하고 있는데 파란색이지만  오류가 나고 있어요당연히...   
228  정렬 기능을 사용중인데아래의 그림과 같이 2가지의 문제가 해결되지 않고 있습니다 ....   
229  버블에서 아래 UI처럼 +버튼을 누르면 하위에 계속해서 폴더가 추가되도록 프론트를 ...   
230  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
231  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                 sent2  label  sent2_index  
0    저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...      1            0  
1    헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...      1            2  
2    모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 ...      1           

In [115]:
#파인튜닝 데이터셋에서 n배의 negative sample을 생성하는 코드
def create_negative_sample(DataFrame, n):
  DataFrame_copy = DataFrame.copy() #원본 데이터셋을 변화시키지 않기 위해 deep copy
  length = len(DataFrame_copy)
  for i in range(length):
    row = DataFrame_copy.iloc[i]
    negative_candidate_ids = list(set(DataFrame_copy['sent2_index'].tolist())) #.remove(row['sent2_index']) #해당 행의 sent2를 negative sample 후보군에서 제외
    negative_candidate_ids.remove(row['sent2_index'])
    negative_sample_ids = list(np.random.choice(negative_candidate_ids, n, replace = False))
    for idx in negative_sample_ids:
      new_row = pd.Series({'sent1' : row['sent1'], 'sent2' : np.random.choice(DataFrame_copy[DataFrame_copy['sent2_index'] == idx]['sent2'].tolist(), 1)[0], 'label' : 0, 'sent2_index' : idx}) #카톡 데이터는 idx 중복이 없기 때문에 랜덤추출로 1개를 뽑아도 달라지는 건 없고, 커뮤니티 데이터의 경우 idx 중복이 있기 때문에 해당 idx를 가진 sent2 중 하나를 뽑아야 함.
      DataFrame_copy = pd.concat([DataFrame_copy, new_row.to_frame().T], ignore_index = True)
  return DataFrame_copy

In [116]:
#커뮤니티 파인튜닝 데이터셋에서 negative sample 생성
df_community_finetune_negative_sampled = create_negative_sample(df_community_finetune, 7)
df_community_finetune_negative_sampled = df_community_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_community_finetune_negative_sampled))
print(df_community_finetune_negative_sampled)

1856
                                                  sent1  \
0     안녕하세요! 우선 모두의노코드 사이트 런칭을 축하드립니다.이렇게 첫번째 질문 글을 ...   
1     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
2     안녕하세요.모두의 노코드 런칭을 축하드립니다. 앞으로 좋은 활동 기대합니다. 현재 ...   
3     안녕하세요.제주도에서 노션으로 동네 친구 매칭 서비스를 만들고 있습니다.어제 웨비나...   
4     모두의노코드 만드신 분들께 질문 드리고 싶어요.제가 버블로 만든 서비스 관련해 모바...   
...                                                 ...   
1851  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1852  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1853  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1854  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   
1855  안녕하세요! 현재 사업을 진행하면서 앱 서비스를 메인 비즈니스로 잡고 진행하려고 해...   

                                                  sent2 label  
0     저도 궁금해져서 열심히 찾아봤는데 콜아웃 같은 블록을 넣는 명령어나 함수는 찾지 못...     1  
1     헤더그룹 안에 로고와 CTA를 넣으시고 max width를 지정하시면 될 것 같아요...     1  
2     모두의노코드를 여쭤보신 것인지 모르겠어서 이게 맞는지는 모르겠습니다.모두의노코드의 ...     1  
3     충분히 버블로 개발가능할 것 같아요! 이미 

In [117]:
print(df_talk_finetune)

                                                 sent1  \
0     deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1     유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...   
2     혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
3     혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
4     혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
..                                                 ...   
496  각 element에다가 condition을 따로 쓰면 workflow에 안들어가있기...   
497  각 element에다가 condition을 따로 쓰면 workflow에 안들어가있기...   
498   안녕하세요, 혹시 버블에서 three.js라는 라이브러리 연결을 도와주실 분 계실...   
499   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
500   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                 sent2  label  sent2_index  
0     익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...      1            1  
1     이게 근데 한번 락인되니까 DB 이관한다는 선택지를 고르기가 참 어렵네요.. 또 ...      0           13  
2     1. Do a search for 할 때 constraints 걸어서 검색량 줄이...      1           

In [118]:
#카카오톡 파인튜닝 데이터셋에서 negative sample 생성
df_talk_finetune_negative_sampled = create_negative_sample(df_talk_finetune, 7)
df_talk_finetune_negative_sampled = df_talk_finetune_negative_sampled.drop('sent2_index', axis = 1)
print(len(df_talk_finetune_negative_sampled))
print(df_talk_finetune_negative_sampled)

4000
                                                  sent1  \
0      deltaPercent의 절대값이 큰 값부터 데이터를 정렬하고 싶은데.deltaP...   
1      유저의 input url에 따른 rss 피드 긁어오기를 하고 싶은데 헷갈리는 부분...   
2      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
3      혹시 버블 워크로드 최적화하는 팁이 있을까요? 아직 레거시 요금제 쓰고 있긴 한데...   
4      혹시 어떤 product 이신가요? 와...이러면 버블 못 쓸 거 같은데요 ㅡㅡ;;;;   
...                                                 ...   
3995   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3996   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3997   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3998   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   
3999   안녕하세요 버블 고수님들!버블 데이터 처리 관련 질문 있습니다!기존에 있는 데이터...   

                                                  sent2 label  
0      익스프레션 내에서 조건식을 사용할 수가 없어서 condition을 사용하는 수밖에...     1  
1      이게 근데 한번 락인되니까 DB 이관한다는 선택지를 고르기가 참 어렵네요.. 또 ...     0  
2      1. Do a search for 할 때 constraints 걸어서 검색량 줄이...     1  
3                    Do a sear

In [119]:
#두 파인튜닝 데이터셋을 하나로 합치기
df_KcBERT_finetune = pd.concat([df_community_finetune_negative_sampled, df_talk_finetune_negative_sampled], ignore_index = True)
print(len(df_KcBERT_finetune))

5856


In [120]:
#.xlsx 파일로 저장
finetune_data_save_path = '/content/drive/My Drive/KcBERT_finetune_dataset.xlsx'

df_KcBERT_finetune.to_excel(finetune_data_save_path)